CNN LSTM 

Data Set 14
Using data zero data zero padding


In [1]:
%load_ext tensorboard
import tensorflow as tf
import datetime

import numpy as np
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input ,Dense, Dropout, Activation, LSTM, Conv1D
from keras.layers import Lambda, Convolution1D, MaxPooling1D, Flatten, Reshape, BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.metrics import categorical_crossentropy, binary_crossentropy

#from spp.SpatialPyramidPooling import SpatialPyramidPooling

#For data saving
import pickle
import random
#other imports
import gzip
import glob
import os
import keras.backend as K
import os
import time 
#cwd = os.path.dirname(os.path.realpath("SURF_001_TwoClass13.ipynb"))

In [2]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

!rm -rf ./logs/ 

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

**Preparing inputs**

In [7]:
with open('Version_e/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_labels_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  train_labels = pickle.load(f)
with open('Version_f/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_samples_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  train_samples = pickle.load(f)
print(train_labels.shape)
print(train_samples.shape)

with open('Version_e/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_labels_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  test_labels = pickle.load(f)
with open('Version_f/' + str(int(0 / 20000))+'/' + str(int(0 / 100))+'/DataSet_14_train_samples_onehot_batches1_Part_' + str(1) + '.dat', 'rb') as f:
  test_samples = pickle.load(f)
print(test_labels.shape)
print(test_samples.shape)


'''

with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_labels_onehot_Part:' + str(1) + '.dat', 'rb') as f:
  train_labels = pickle.load(f)
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_samples_onehot_Part:' + str(1) + '.dat', 'rb') as f:
  train_samples = pickle.load(f)
'''

(1, 2)
(1, 10000, 4)
(1, 2)
(1, 10000, 4)


"\n\nwith open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_labels_onehot_Part:' + str(1) + '.dat', 'rb') as f:\n  train_labels = pickle.load(f)\nwith open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_train_samples_onehot_Part:' + str(1) + '.dat', 'rb') as f:\n  train_samples = pickle.load(f)\n"

In [ ]:
print(train_labels)
print(train_labels.shape)


[1 0]
(2,)


CNN LSTM Model:

In [8]:
dense_layer = 1
layer_size = 64 
conv_layer = 2
seq_length = 1000
base_pair = 4
num_strides = 1
pool_size = 4

Notes:

Seems to crash when data is stored in batches of 200 and largerst seq length 310400




In [9]:

# input  images of 4 by 200
model = Sequential()
model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
model.add(Conv1D(layer_size, 4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))

model.add(Dropout(0.1, noise_shape=None, seed=None))

for l in range(conv_layer-1):
  model.add(MaxPooling1D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
  model.add(Conv1D(layer_size, 4, activation='relu'))

#model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last'))
#model.add(Conv1D(64, 4, activation='relu'))

model.add(Dropout(0.1, noise_shape=None, seed=None))

model.add(LSTM(32))

model.add(Flatten())

for l in range(dense_layer):
  model.add(Dense(12))
  model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=10)

'''
odel = Sequential()
model.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))
model.add(MaxPooling1D(pool_size=4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', data_format='channels_last'))
model.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))
model.add(Dropout(0.1, noise_shape=None, seed=None))

for l in range(conv_layer-1):
  model.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))
  model.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))
  model.add(Dropout(0.1, noise_shape=None, seed=None))

#model.add(LSTM(64))


model.add(Flatten())

#for l in range(dense_layer):
#  model.add(Dense(64))
#  model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
#callbacks=[]
model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=3)
model.save('gdrive/My Drive/SURF_2020_Weiss/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-200-epochs-8-version-Part:1-CNNLSTM'.format(conv_layer, layer_size, dense_layer))
#version 3 is with all the chromosomes
'''

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 997, 64)           1088      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 994, 64)           16448     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 248, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 248, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 59, 64)            16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 64)            0

"\nodel = Sequential()\nmodel.add(Conv1D(filters=layer_size, kernel_size=(4), input_shape=(seq_length, 4), activation='relu', strides=1, padding='valid'))\nmodel.add(MaxPooling1D(pool_size=4, batch_input_shape=(1, seq_length, 4), strides=1, padding='valid', data_format='channels_last'))\nmodel.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))\nmodel.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))\nmodel.add(Dropout(0.1, noise_shape=None, seed=None))\n\nfor l in range(conv_layer-1):\n  model.add(Conv1D(layer_size, 4, strides=1, padding='valid', activation='relu'))\n  model.add(MaxPooling1D(pool_size=4, strides=1, padding='valid', data_format='channels_last'))\n  model.add(Dropout(0.1, noise_shape=None, seed=None))\n\n#model.add(LSTM(64))\n\n\nmodel.add(Flatten())\n\n#for l in range(dense_layer):\n#  model.add(Dense(64))\n#  model.add(Activation('relu'))\nmodel.add(Dense(2, activation='softmax'))\n\nmodel.compile(optimizer='

In [ ]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
num_place = 2

#1 - 2411
for part in range(3, 54300):
  num_place += 1
  if num_place > 20000:
    num_place -= 20000
  if part != 1:
    with open('gdrive/My Drive/SURF_2020_Weiss/Data_Sets/DataSet13/test5/' + str(int(part / 20000))+'/' + str(int(num_place / 100))+'/DataSet_13_train_labels_onehot_batches5_Part:' + str(part) + '.dat', 'rb') as f:
      train_labels = pickle.load(f)
    with open('gdrive/My Drive/SURF_2020_Weiss/Data_Sets/DataSet13/test5/' + str(int(part / 20000))+'/' + str(int(num_place / 100))+'/DataSet_13_train_samples_onehot_batches5_Part:' + str(part) + '.dat', 'rb') as f:
      train_samples = pickle.load(f)

    shuffle = np.random.permutation(len(train_samples))
    train_samples = train_samples[shuffle]
    train_labels = train_labels[shuffle]
    
    model.fit(train_samples, train_labels, validation_data=(test_samples, test_labels), epochs=3)
    train_samples = []
    train_labels = []
    if part % 100 == 0 or part == 50:
      print(model.summary())
      print('Part:' + str(part))
      model.save('gdrive/My Drive/SURF_2020_Weiss/008_Model/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-zeroPadding-3-epochs-2-version-Part:{}-CNNLSTM'.format(conv_layer, layer_size, dense_layer, part))
# Versions for Model 008
# Version 1: 1 bat

Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 0.6832 - accuracy: 1.0000 - val_loss: 0.6822 - val_accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.6822 - accuracy: 1.0000 - val_loss: 0.6812 - val_accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 2s 2s/step - loss: 0.6812 - accuracy: 1.0000 - val_loss: 0.6802 - val_accuracy: 1.0000
Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 0.6802 - accuracy: 1.0000 - val_loss: 0.6793 - val_accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.6793 - accuracy: 1.0000 - val_loss: 0.6783 - val_accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 2s 2s/step - loss: 0.6783 - accuracy: 1.0000 - val_loss: 0.6773 - val_accuracy: 1.0000
Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 0.6773 - accuracy: 1.0000 - val_loss: 0.6763 - val_accuracy: 1.0000
Epoch 2/3
1/1 [==============================] -

**Versions:**

Version 5:
  - Issue not loading data
  - Data set 13, data: onehot version with batches of 1, model: batches of 1

Version 6:
 - Got ride of loading, saving, and compiling model each time, ran with data set 13, data: onehot version with batches of 1, model: batches of 1

In [ ]:
%tensorboard --logdir logs
#%tensorboard --logdir logs/fit
#%tensorboard --logdir gdrive/My Drive/SU
#RF_2020_Weiss/Models/E003_CNN_2category

In [ ]:
model.save('gdrive/My Drive/SURF_2020_Weiss/008_Model/Models_H5/008_DataSet13_{}-conv-{}-nodes-{}-dense-zeroPadding-3-epochs-2-version-Part:{}-CNNLSTM'.format(conv_layer, layer_size, dense_layer, part))


NameError: ignored

Testing the model:


In [ ]:
# Loading testing data: 
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_samples_onehot_200bp' + genome[13] + '.dat', 'rb') as f:
  test_samples = pickle.load(f)
  print(test_samples.shape)
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_labels_onehot_200bp' + genome[13] + '.dat', 'rb') as f:
  test_labels = pickle.load(f)
  print(test_labels.shape) 

(120074, 200, 4)
(120074, 11)


In [ ]:
# Load a model
model = load_model('gdrive/My Drive/SURF_2020_Weiss/Models_H5/006_DataSet10_2-conv-64-nodes-0-dense-200-epochs-3-version-chrX-CNNLSTM')

In [ ]:
predicted = model.predict_classes(test_samples)
print(len(predicted))
print(predicted[1000:1100])

120074
[10 10 10  8 10 10  8 10 10 10 10 10  8  8 10 10 10 10  8 10 10 10 10 10
  8  8  8 10 10 10 10 10 10 10 10 10  8 10 10 10 10 10 10 10  8 10 10 10
 10 10 10  8 10 10 10  8  8  8 10 10 10 10 10 10 10  8 10  8 10 10 10 10
  8 10  8 10 10 10 10 10 10 10 10  8  8  8 10  8 10 10 10 10  8 10  8  8
 10 10 10 10]


In [ ]:
expected = []
for label in test_labels:
  for i, num in enumerate(label):
    if num == 1:
      expected.append(i)
count = 0 
count_2 = 0
for x in predicted:
  if x != 0:
    count += 1
for y in expected:
  if y != 0:
    count_2 += 1
print(count)
print(count_2)

120074
119450


In [ ]:
print(expected[1000:1100])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
count = 0
for i, num in enumerate(expected):
  if num == 1 and predicted[i] == 1: 
    count += 1
print(count)

0


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

results_2 = multilabel_confusion_matrix(expected, predicted)
print(results_2)

[[[119450      0]
  [   624      0]]

 [[119316      0]
  [   758      0]]

 [[119492      0]
  [   582      0]]

 [[119154      0]
  [   920      0]]

 [[119832      0]
  [   242      0]]

 [[119912      0]
  [   162      0]]

 [[119454      0]
  [   620      0]]

 [[119198      0]
  [   876      0]]

 [[118943    371]
  [   650    110]]

 [[117176      0]
  [  2898      0]]

 [[   336   8106]
  [   145 111487]]]


In [ ]:
testing = [1, 2, 3, 3]
testing_2 = [1, 2, 3 , 3]
results_2 = multilabel_confusion_matrix(testing, testing_2)
print(results_2)

Extra Code:


In [ ]:
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_samples_onehot_200bp' + genome[7] + '.dat', 'rb') as f:
  test_samples = pickle.load(f)
  print(test_samples.shape)
with open('gdrive/My Drive/SURF_2020_Weiss/E003_hg38_25_imputed12marks/DataSet_13_test_labels_onehot_200bp' + genome[7] + '.dat', 'rb') as f:
  test_labels = pickle.load(f)
  print(test_labels.shape) 

In [ ]:
predicted = model.predict_classes(test_labels)
print(len(predicted))
print(predicted[0])

In [ ]:
expected = []
for label in test_labels:
  for i, num in enumerate(label):
    if num == 1:
      expected.append(i)

In [ ]:

# Example of a confusion matrix in Python
from sklearn.metrics import confusion_matrix
 
results = confusion_matrix(expected, predicted)
print(results)
# Printing the precision and recall, among other metrics
#print(metrics.classification_report(y_act, y_pred, labels=["a", 
#"b"]))

[[4 2]
 [1 3]]


In [ ]:
train_samples.shape

(131148, 200, 4)

In [ ]:
train_labels.shape
print(train_labels[0][0])
train_labels_flat = []
for i in range(0, len(train_labels)):
  train_labels_flat.append(train_labels[i][0])
print(len(train_labels))

test_labels.shape
print(test_labels[0][0])
test_labels_flat = []
for i in range(0, len(test_labels)):
  test_labels_flat.append(test_labels[i][0])
print(len(test_labels))

0.0
131148
0.0
30073


In [ ]:
count = 0
for i in test_labels:
  if i[0] == 1:
    count += 1

print(count)

3761


In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_samples, test_samples, train_labels_flat, test_labels_flat

nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))

#nsamples, nx, ny = y_train.shape
#y_train = y_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))

#nsamples, nx, ny = y_test.shape
#y_test = y_test.reshape((nsamples,nx*ny))

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

Automatically created module for IPython interactive environment


NameError: ignored